# Notebook for running stairwayplot2 on eastern snake data

This is modified from a notebook for the same purpose from Dr. Arianna Kuhn

In [1]:
# import some stuff
import glob
import ipyparallel as ipp
import os
import subprocess

# For this to work, dadi must be installed
# I run this from my easySFS conda environment - has dadi installed - check the current environment
!conda env list

easySFS = "/Users/harrington/easySFS/easySFS.py"
# run easySFS and test that it is installed & spits out the options
!$easySFS -h

# conda environments:
#
base                     /Users/harrington/opt/miniconda3
bioinfo                  /Users/harrington/opt/miniconda3/envs/bioinfo
easySFS               *  /Users/harrington/opt/miniconda3/envs/easySFS
entropy                  /Users/harrington/opt/miniconda3/envs/entropy
fastqc                   /Users/harrington/opt/miniconda3/envs/fastqc
gffread                  /Users/harrington/opt/miniconda3/envs/gffread
ipyrad                   /Users/harrington/opt/miniconda3/envs/ipyrad
macs2                    /Users/harrington/opt/miniconda3/envs/macs2
pixy                     /Users/harrington/opt/miniconda3/envs/pixy
py-popgen                /Users/harrington/opt/miniconda3/envs/py-popgen
qiime2-2022.2            /Users/harrington/opt/miniconda3/envs/qiime2-2022.2
splitfasta               /Users/harrington/opt/miniconda3/envs/splitfasta

usage: easySFS.py [-h] [-a] -i VCF_NAME -p POPULATIONS [--proj PROJECTIONS]
                  [--preview] [-o OUTDIR] [--ploidy PLOI

In [2]:
# Set up some directories
analysis_dir = "/Users/harrington/Active_Research/Ecotone_genomics/GBS_Data/stairwayplot2/analysis/"
pops_dir = "/Users/harrington/Active_Research/Ecotone_genomics/GBS_Data/Coexp_dryad/stairwayplot2/pop_files/"
vcf_dir = "/Users/harrington/Active_Research/Ecotone_genomics/GBS_Data/Coexp_dryad/stairwayplot2/filtered_vcf/"

# Populations files for each pop were made with the "make_pops_SFS_stairway.R" script
all_pops = glob.glob(pops_dir + "*")
all_pops = sorted([x.split("/")[-1].rsplit(".", 1)[0] for x in all_pops])

all_pops

['Acontortrix_p123_v2_25miss_popAcontortrix',
 'Acontortrix_p123_v2_25miss_popeast',
 'Acontortrix_p123_v2_25miss_popwest',
 'Dpunctatus_p123_v3_25missEAST_popDpunctatus',
 'Dpunctatus_p123_v3_25missEAST_popcentral',
 'Dpunctatus_p123_v3_25missEAST_popnorth',
 'Dpunctatus_p123_v3_25missEAST_popsouth',
 'Lgetula_p123_v4_25miss_popcent',
 'Lgetula_p123_v4_25miss_popeast',
 'Lgetula_p123_v4_25miss_popholb',
 'Mflagellum_p123_v3_25missEast_popMflagellum',
 'Pguttatus_p123_v2_25miss_popemor',
 'Pguttatus_p123_v2_25miss_popgut',
 'Sdekayi_p123_v4_25miss_popSdekayi',
 'abacura_only_popeast',
 'abacura_only_popwest',
 'erytro_poperytro',
 'milks_denovo-92_popelap',
 'milks_denovo-92_popgent',
 'milks_denovo-92_poptri']

In [3]:
## prototype the easySFS preview call for one population
pop = "Sdekayi_p123_v4_25miss_popSdekayi" # define the population
in_vcf = vcf_dir + pop + ".vcf" # define the vcf file location
pop_file = pops_dir + pop + ".txt" # define the population file location
print(in_vcf)
print(pop_file)

!$easySFS -i "$in_vcf" -p "$pop_file" -a --preview # run easySFS preview

/Users/harrington/Active_Research/Ecotone_genomics/GBS_Data/Coexp_dryad/stairwayplot2/filtered_vcf/Sdekayi_p123_v4_25miss_popSdekayi.vcf
/Users/harrington/Active_Research/Ecotone_genomics/GBS_Data/Coexp_dryad/stairwayplot2/pop_files/Sdekayi_p123_v4_25miss_popSdekayi.txt
Processing 1 populations - odict_keys(['Sdekayi'])

    Running preview mode. We will print out the results for # of segregating sites
    for multiple values of projecting down for each population. The dadi
    manual recommends maximizing the # of seg sites for projections, but also
    a balance must be struck between # of seg sites and sample size.

    For each population you should choose the value of the projection that looks
    best and then rerun easySFS with the `--proj` flag.
    
Sdekayi
(2, 4317)	(3, 6423)	(4, 8135)	(5, 9606)	(6, 10965)	(7, 12201)	(8, 13383)	(9, 14463)	(10, 15530)	(11, 16532)	(12, 17518)	(13, 18456)	(14, 19379)	(15, 20259)	(16, 21131)	(17, 21974)	(18, 22805)	(19, 23608)	(20, 24402)	(21, 25

In [4]:
# Get easySFS preview results for all populations
preview_dict = {}
for pop in all_pops:
    print(pop)
    in_vcf = vcf_dir + pop + ".vcf"
    pop_file = pops_dir + pop + ".txt"
    preview_dict[pop] = !$easySFS -i "$in_vcf" -p "$pop_file" -a --preview

Acontortrix_p123_v2_25miss_popAcontortrix
Acontortrix_p123_v2_25miss_popeast
Acontortrix_p123_v2_25miss_popwest
Dpunctatus_p123_v3_25missEAST_popDpunctatus
Dpunctatus_p123_v3_25missEAST_popcentral
Dpunctatus_p123_v3_25missEAST_popnorth
Dpunctatus_p123_v3_25missEAST_popsouth
Lgetula_p123_v4_25miss_popcent
Lgetula_p123_v4_25miss_popeast
Lgetula_p123_v4_25miss_popholb
Mflagellum_p123_v3_25missEast_popMflagellum
Pguttatus_p123_v2_25miss_popemor
Pguttatus_p123_v2_25miss_popgut
Sdekayi_p123_v4_25miss_popSdekayi
abacura_only_popeast
abacura_only_popwest
erytro_poperytro
milks_denovo-92_popelap
milks_denovo-92_popgent
milks_denovo-92_poptri


In [5]:
# Show the easySFS preview results for all pops
for k, v in preview_dict.items():
    print(k, "\n", v[-3], "\n")

Acontortrix_p123_v2_25miss_popAcontortrix 
 (2, 6528)	(3, 9771)	(4, 12355)	(5, 14585)	(6, 16629)	(7, 18502)	(8, 20313)	(9, 21991)	(10, 23667)	(11, 25266)	(12, 26853)	(13, 28392)	(14, 29918)	(15, 31397)	(16, 32875)	(17, 34307)	(18, 35747)	(19, 37142)	(20, 38551)	(21, 39933)	(22, 41317)	(23, 42668)	(24, 44030)	(25, 45355)	(26, 46697)	(27, 47997)	(28, 49322)	(29, 50609)	(30, 51917)	(31, 53171)	(32, 54465)	(33, 55686)	(34, 56967)	(35, 58163)	(36, 59431)	(37, 60609)	(38, 61865)	(39, 62999)	(40, 64244)	(41, 65237)	(42, 66470)	(43, 67119)	(44, 68337)	(45, 68133)	(46, 69331)	(47, 66878)	(48, 68026)	(49, 51807)	(50, 52660)	(51, 38546)	(52, 39158)	(53, 27102)	(54, 27514)	(55, 17960)	(56, 18226)	(57, 10641)	(58, 10793)	(59, 5530)	(60, 5607)	(61, 2182)	(62, 2210)	(63, 563)	(64, 570)	 

Acontortrix_p123_v2_25miss_popeast 
 (2, 6383)	(3, 9563)	(4, 12085)	(5, 14242)	(6, 16230)	(7, 18040)	(8, 19803)	(9, 21454)	(10, 23092)	(11, 24646)	(12, 26203)	(13, 27684)	(14, 29185)	(15, 30600)	(16, 32057)	(17, 333

In [ ]:
# Set projection level for each population. Have to choose the proj level
# based on the sampling.....
# For stairwayplot 14 or more samples give nice looking output sfs plots - but don't always have this level of sampling

proj_dict = {}
proj_dict["Acontortrix_p123_v2_25miss_popAcontortrix"] = 46 # all A. contortrix combined

proj_dict["Acontortrix_p123_v2_25miss_popeast"] = 24
proj_dict["Acontortrix_p123_v2_25miss_popwest"] = 22

proj_dict["Dpunctatus_p123_v3_25missEAST_popDpunctatus"] = 52 # all D. punctatus combined

proj_dict["Dpunctatus_p123_v3_25missEAST_popcentral"] = 28
proj_dict["Dpunctatus_p123_v3_25missEAST_popnorth"] = 22
proj_dict["Dpunctatus_p123_v3_25missEAST_popsouth"] = 6

proj_dict["Lgetula_p123_v4_25miss_popeast"] = 6
proj_dict["Lgetula_p123_v4_25miss_popholb"] = 30
proj_dict["Lgetula_p123_v4_25miss_popcent"] = 14
proj_dict["Mflagellum_p123_v3_25missEast_popMflagellum"] = 14
proj_dict["Pguttatus_p123_v2_25miss_popemor"] = 30
proj_dict["Pguttatus_p123_v2_25miss_popgut"] = 24
proj_dict['Sdekayi_p123_v4_25miss_popSdekayi'] = 40
proj_dict["abacura_only_popeast"] = 44
proj_dict["abacura_only_popwest"] = 20
proj_dict["erytro_poperytro"] = 23
proj_dict["milks_denovo-92_popelap"] = 42
proj_dict["milks_denovo-92_poptri"] = 50
proj_dict["milks_denovo-92_popgent"] = 42





In [ ]:
## prototype the easySFS proj call for one population
pop = "Sdekayi_p123_v4_25miss_popSdekayi"
pop_analysis_dir = analysis_dir + pop + "/sfs"
in_vcf = vcf_dir + "{pop}.vcf".format(pop=pop)
pop_file = pops_dir + "{}.txt".format(pop)
if not os.path.exists(pop_analysis_dir):
    os.makedirs(pop_analysis_dir)
    
!$easySFS -i "$in_vcf" -p "$pop_file" -a --proj 46 -o "$pop_analysis_dir" -f

In [ ]:
# Get easySFS proj results for all populations
preview_dict = {}
for pop in all_pops:
    print(pop)
    in_vcf = vcf_dir + "{pop}.vcf".format(pop=pop)
    pop_file = pops_dir + "{}.txt".format(pop)
    pop_analysis_dir = analysis_dir + pop + "/sfs"
    if not os.path.exists(pop_analysis_dir):
        os.makedirs(pop_analysis_dir)
    proj = proj_dict[pop]
    !$easySFS -i "$in_vcf" -p "$pop_file" -a --proj $proj -o "$pop_analysis_dir" -f

### make blueprint files for each population

here, using same rates as I did in the C. ruber paper, plus generation time of 3 years:

2.2*10-9 / year

assume generation time of 3 years

->> 6.6e-9 per generation
   

In [ ]:
blueprint_template = """#example blueprint file
#input setting
popid: {pop} # id of the population (no white space)
nseq: {nseqs} # number of sequences
L: {seq_length} # total number of observed nucleic sites, including polymorphic and monomorphic
whether_folded: true # whethr the SFS is folded (true or false)
SFS: {sfs} # snp frequency spectrum: number of singleton, number of doubleton, etc. (separated by white space)
#smallest_size_of_SFS_bin_used_for_estimation: 1 # default is 1; to ignore singletons, uncomment this line and change this number to 2
#largest_size_of_SFS_bin_used_for_estimation: 15 # default is nseq/2 for folded SFS
pct_training: 0.67 # percentage of sites for training
nrand: 7 15 22 28 # number of random break points for each try (separated by white space)
project_dir: '/Users/harrington/Active_Research/Ecotone_genomics/GBS_Data/stairwayplot2/analysis/{pop}/{nseqs}_samples' # project directory
stairway_plot_dir: /Applications/stairway_plot_v2.1.1/stairway_plot_es # directory to the stairway plot files
ninput: 200 # number of input files to be created for each estimation
#random_seed: 666
#output setting
mu: 6.6e-9 # assumed mutation rate per site per generation
year_per_generation: 3 # assumed generation time (in years)
#plot setting
plot_title: {pop} # title of the plot
xrange: 0.1,10000 # Time (1k year) range; format: xmin,xmax; "0,0" for default
yrange: 0,0 # Ne (1k individual) range; format: xmin,xmax; "0,0" for default
xspacing: 2 # X axis spacing
yspacing: 2 # Y axis spacing
fontsize: 12 # Font size"""

In [ ]:
stairwayplot_es_dir = "/Applications/stairway_plot_v2.1.1/stairway_plot_es"

# prototype for one population
pop = "Sdekayi_p123_v4_25miss_popSdekayi"
# get the number of samples retained in the projection
nsamps = proj_dict[pop] 
# get the full sequence length from the stats file
in_stats = vcf_dir + pop.rsplit("_", 1)[0] + "_stats.txt"
seq_length = open(in_stats).readlines()[-1].split(",")[1].strip().split(")")[0]


# get the folded sfs
pop_sfs = analysis_dir + pop + "/sfs/dadi/" + pop.split("_pop", 1)[1] + ".sfs".format(pop=pop)
pop_sfs = open(pop_sfs).readlines()[1].split()[1:int(nsamps/2)+1]

# Write the blueprint file
blueprint_file = analysis_dir + "{pop}/{pop}.blueprint".format(pop=pop)
with open(blueprint_file, 'w') as outfile:
    outfile.write(blueprint_template.format(pop=pop, nseqs=nsamps, seq_length=seq_length, sfs=" ".join(pop_sfs)))

# This call creates $blueprint_file.sh which gets run
!java -cp $stairwayplot_es_dir Stairbuilder "$blueprint_file"
stairwayplot2_script = blueprint_file + ".sh"
!bash "$stairwayplot2_script"

In [ ]:
## do them all in a serial fashion, dumb but easy

for pop in all_pops:
    print(pop)
    stairwayplot_es_dir = "/Applications/stairway_plot_v2.1.1/stairway_plot_es"
    # Get the number of samples retained in the projection
    nsamps = proj_dict[pop]
    # get the full sequence length from the stats file
    in_stats = vcf_dir + pop.rsplit("_", 1)[0] + "_stats.txt"
    seq_length = open(in_stats).readlines()[-1].split(",")[1].strip().split(")")[0]

    # get the folded sfs
    pop_sfs = analysis_dir + pop + "/sfs/dadi/" + pop.split("_pop", 1)[1] + ".sfs".format(pop=pop)
    pop_sfs = open(pop_sfs).readlines()[1].split()[1:int(nsamps/2)+1]

    # Write the blueprint file
    blueprint_file = analysis_dir + "{pop}/{pop}.blueprint".format(pop=pop)
    with open(blueprint_file, 'w') as outfile:
        outfile.write(blueprint_template.format(pop=pop, nseqs=nsamps, seq_length=seq_length, sfs=" ".join(pop_sfs)))
    
    # This call creates $blueprint_file.sh which gets run
    _ = !java -cp $stairwayplot_es_dir Stairbuilder "$blueprint_file"
    stairwayplot2_script = blueprint_file + ".sh"
    _ = !bash "$stairwayplot2_script"